In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 22s]
val_accuracy: 0.8613333106040955

Best val_accuracy So Far: 0.9120000004768372
Total elapsed time: 00h 07m 12s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 80)        800       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        46144     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,996,954
Trainable params: 2,996,954
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 20s 11ms/step - loss: 0.1187 - accuracy: 0.9546 - val_loss: 0.2653 - val_accuracy: 0.9087
Epoch 5/10
1688/1688 [==============================] - 19s 11ms/step - loss: 0.0764 - accuracy: 0.9715 - val_loss: 0.2986 - val_accuracy: 0.9118
Epoch 6/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.0534 - accuracy: 0.9803 - val_loss: 0.3612 - val_accuracy: 0.9130
Epoch 7/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0382 - accuracy: 0.9863 - val_loss: 0.4179 - val_accuracy: 0.9135
Epoch 8/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0291 - accuracy: 0.9896 - val_loss: 0.4673 - val_accuracy: 0.9147
Epoch 9/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0254 - accuracy: 0.9908 - val_loss: 0.5069 - val_accuracy: 0.9152
Epoch 10/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0208 - accuracy: 0.9927 - val_loss: 0.5628 